In [15]:
import cv2
import numpy as np
from scipy import ndimage
import scipy
from scipy import signal
from scipy import ndimage
import imageio
import time
import numpy as np

import matplotlib.pyplot as plt

In [6]:


start = time.time()
# outframe = hdmi_out.newframe()
#2D Sobel operator https://stackoverflow.com/questions/7185655/applying-the-sobel-filter-using-scipy
im = imageio.imread("cartoon.jpg")
# im = cv2.imread("cartoon.jpg")
im = im.astype('int32')
dx = ndimage.sobel(im, 0) # horizontal deriv
dy = ndimage.sobel(im, 1) # vertical deriv
mag = np.hypot(dx, dy) # magnitude
mag *= 255.0/np.max(mag) #normalize
cv2.imwrite("cartoon_mag.jpg", mag)

end = time.time()

print(end-start)

0.04377436637878418


In [8]:
start = time.time()

# https://medium.com/@nikatsanka/comparing-edge-detection-methods-638a2919476e

# sobel operator manually

# Apply gray scale
# gray_img = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
gray_img = np.round(0.299 * im[:, :, 0] +
                    0.587 * im[:, :, 1] +
                    0.114 * im[:, :, 2]).astype(np.uint8)

# Sobel Operator
h, w = gray_img.shape
# define filters
horizontal = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])  # s2
vertical = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])  # s1

# define images with 0s
newhorizontalImage = np.zeros((h, w))
newverticalImage = np.zeros((h, w))
newgradientImage_sobel = np.zeros((h, w))

# offset by 1
for i in range(1, h - 1):
    for j in range(1, w - 1):
        horizontalGrad = (horizontal[0, 0] * gray_img[i - 1, j - 1]) + \
                         (horizontal[0, 1] * gray_img[i - 1, j]) + \
                         (horizontal[0, 2] * gray_img[i - 1, j + 1]) + \
                         (horizontal[1, 0] * gray_img[i, j - 1]) + \
                         (horizontal[1, 1] * gray_img[i, j]) + \
                         (horizontal[1, 2] * gray_img[i, j + 1]) + \
                         (horizontal[2, 0] * gray_img[i + 1, j - 1]) + \
                         (horizontal[2, 1] * gray_img[i + 1, j]) + \
                         (horizontal[2, 2] * gray_img[i + 1, j + 1])

        newhorizontalImage[i - 1, j - 1] = abs(horizontalGrad)

        verticalGrad = (vertical[0, 0] * gray_img[i - 1, j - 1]) + \
                       (vertical[0, 1] * gray_img[i - 1, j]) + \
                       (vertical[0, 2] * gray_img[i - 1, j + 1]) + \
                       (vertical[1, 0] * gray_img[i, j - 1]) + \
                       (vertical[1, 1] * gray_img[i, j]) + \
                       (vertical[1, 2] * gray_img[i, j + 1]) + \
                       (vertical[2, 0] * gray_img[i + 1, j - 1]) + \
                       (vertical[2, 1] * gray_img[i + 1, j]) + \
                       (vertical[2, 2] * gray_img[i + 1, j + 1])

        newverticalImage[i - 1, j - 1] = abs(verticalGrad)

        # Edge Magnitude
        mag = np.sqrt(pow(horizontalGrad, 2.0) + pow(verticalGrad, 2.0))
        newgradientImage_sobel[i - 1, j - 1] = mag

cv2.imwrite("cartoon_sobel.jpg", newgradientImage_sobel)

end = time.time()

print(end-start)


3.2597639560699463


In [53]:
#https://stackoverflow.com/questions/56357039/numpy-zero-padding-to-match-a-certain-shape
def to_shape(a, shape):
    b = np.zeros(shape)
    b[:a.shape[0], :a.shape[1]] = a
    return b

img_fft = np.fft.fft2(gray_img)
horizontal_kernel_fft =  np.fft.fft2(to_shape(horizontal, gray_img.shape))
vertical_kernel_fft =  np.fft.fft2(to_shape(vertical, gray_img.shape) )


newverticalImage = np.fft.ifft2( img_fft * vertical_kernel_fft )
newhorizontalImage = np.fft.ifft2( img_fft * horizontal_kernel_fft )

newgradientImage_sobel_fft = abs( np.sqrt(pow(newverticalImage, 2) + pow(newhorizontalImage, 2)) ).astype(float)

print(newgradientImage_sobel)
cv2.imwrite("cartoon_sobel_fft.jpg", newgradientImage_sobel_fft)
cv2.imwrite("cartoon_sobel_diff.jpg", newgradientImage_sobel - newgradientImage_sobel_fft)

def nonmax_suppression(det, phase):
    gmax = np.zeros(det.shape)
    for i in range(gmax.shape[0]):
        for j in range(gmax.shape[1]):
            if phase[i][j] < 0:
                phase[i][j] += 360

            if ((j + 1) < gmax.shape[1]) and ((j - 1) >= 0) and ((i + 1) < gmax.shape[0]) and ((i - 1) >= 0):
                # 0 degrees
                if (phase[i][j] >= 337.5 or phase[i][j] < 22.5) or (phase[i][j] >= 157.5 and phase[i][j] < 202.5):
                    if det[i][j] >= det[i][j + 1] and det[i][j] >= det[i][j - 1]:
                        gmax[i][j] = det[i][j]
                # 45 degrees
                if (phase[i][j] >= 22.5 and phase[i][j] < 67.5) or (phase[i][j] >= 202.5 and phase[i][j] < 247.5):
                    if det[i][j] >= det[i - 1][j + 1] and det[i][j] >= det[i + 1][j - 1]:
                        gmax[i][j] = det[i][j]
                # 90 degrees
                if (phase[i][j] >= 67.5 and phase[i][j] < 112.5) or (phase[i][j] >= 247.5 and phase[i][j] < 292.5):
                    if det[i][j] >= det[i - 1][j] and det[i][j] >= det[i + 1][j]:
                        gmax[i][j] = det[i][j]
                # 135 degrees
                if (phase[i][j] >= 112.5 and phase[i][j] < 157.5) or (phase[i][j] >= 292.5 and phase[i][j] < 337.5):
                    if det[i][j] >= det[i - 1][j - 1] and det[i][j] >= det[i + 1][j + 1]:
                        gmax[i][j] = det[i][j]
    return gmax

def nonmax_suppression_conv(det, phase):
    kernel = { 
        'up': [[0,-1,0],[0,1,0],[0,0,0]],
        'down': [[0,0,0],[0,1,0],[0,-1,0]],
        'left': [[0,0,0],[-1,1,0],[0,0,0]],
        'right': [[0,0,0],[0,1,-1],[0,0,0]],
        'up_left': [[-1,0,0],[0,1,0],[0,0,0]],
        'down_left': [[0,0,0],[0,1,0],[-1,0,0]],
        'up_right': [[0,0,-1],[0,1,0],[0,0,0]],
        'down_right': [[0,0,0],[0,1,0],[0,0,-1]],
    }

    phase[ phase < 0 ] += 360

    output = np.zeros(phase.shape)
    print(phase.shape)

    output = np.logical_and ( np.greater(signal.convolve2d(det, kernel['up'],mode='same'), 0), np.greater(scipy.signal.convolve2d(det, kernel['down'],mode='same'), 0), where=(phase >= 337.5) | (phase < 22.5) | ((phase >= 157.5) & (phase < 202.5)))
    output = np.logical_and ( np.greater(signal.convolve2d(det, kernel['up_left'],mode='same'), 0), np.greater(scipy.signal.convolve2d(det, kernel['down_right'],mode='same'), 0), where=((phase >= 22.5) & (phase < 67.5)) | ((phase >= 202.5) & (phase < 247.5)))
    output= np.logical_and ( np.greater(signal.convolve2d(det, kernel['left'],mode='same'), 0), np.greater(scipy.signal.convolve2d(det, kernel['right'],mode='same'), 0), where=((phase >= 67.5) & (phase < 112.5)) | ((phase >= 247.5) & (phase < 292.5)))
    output = np.logical_and ( np.greater(signal.convolve2d(det, kernel['_right'],mode='same'), 0), np.greater(scipy.signal.convolve2d(det, kernel['up_left'],mode='same'), 0), where=((phase >= 112.5) & (phase < 157.5)) | ((phase >= 292.5) & (phase < 337.5)))

    return np.where(output, det, 0)

angle = np.arctan2(np.real(newverticalImage).astype(float), np.real(newhorizontalImage).astype(float)) * 180 / np.pi
supressed = nonmax_suppression_conv(newgradientImage_sobel,np.arctan2(np.real(newverticalImage).astype(float), np.real(newhorizontalImage).astype(float)) * 180 / np.pi )
print(supressed)
cv2.imwrite("supressed_conv.jpg", supressed)
cv2.imwrite("angles.jpg",angle)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 4. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(640, 640)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


True

In [ ]:

start = time.time()
#prewitt
horizontal = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])  # s2
vertical = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])  # s1

# define images with 0s
newgradientImage_prewitt = np.zeros((h, w))

# offset by 1
for i in range(1, h - 1):
    for j in range(1, w - 1):
        horizontalGrad = (horizontal[0, 0] * gray_img[i - 1, j - 1]) + \
                         (horizontal[0, 1] * gray_img[i - 1, j]) + \
                         (horizontal[0, 2] * gray_img[i - 1, j + 1]) + \
                         (horizontal[1, 0] * gray_img[i, j - 1]) + \
                         (horizontal[1, 1] * gray_img[i, j]) + \
                         (horizontal[1, 2] * gray_img[i, j + 1]) + \
                         (horizontal[2, 0] * gray_img[i + 1, j - 1]) + \
                         (horizontal[2, 1] * gray_img[i + 1, j]) + \
                         (horizontal[2, 2] * gray_img[i + 1, j + 1])

        verticalGrad = (vertical[0, 0] * gray_img[i - 1, j - 1]) + \
                       (vertical[0, 1] * gray_img[i - 1, j]) + \
                       (vertical[0, 2] * gray_img[i - 1, j + 1]) + \
                       (vertical[1, 0] * gray_img[i, j - 1]) + \
                       (vertical[1, 1] * gray_img[i, j]) + \
                       (vertical[1, 2] * gray_img[i, j + 1]) + \
                       (vertical[2, 0] * gray_img[i + 1, j - 1]) + \
                       (vertical[2, 1] * gray_img[i + 1, j]) + \
                       (vertical[2, 2] * gray_img[i + 1, j + 1])

        # Edge Magnitude
        mag = np.sqrt(pow(horizontalGrad, 2.0) + pow(verticalGrad, 2.0))
        newgradientImage_prewitt[i - 1, j - 1] = mag


cv2.imwrite("cartoon_prewitt.jpg", newgradientImage_prewitt)

end = time.time()

print(end-start)

In [ ]:
start = time.time()

#laplachian
horizontal = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]]) #positive laplachian
# horizontal = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]]) #negative laplachian
vertical = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]]) 


# define images with 0s
newgradientImage_laplachian = np.zeros((h, w))

# offset by 1
for i in range(1, h - 1):
    for j in range(1, w - 1):
        horizontalGrad = (horizontal[0, 0] * gray_img[i - 1, j - 1]) + \
                         (horizontal[0, 1] * gray_img[i - 1, j]) + \
                         (horizontal[0, 2] * gray_img[i - 1, j + 1]) + \
                         (horizontal[1, 0] * gray_img[i, j - 1]) + \
                         (horizontal[1, 1] * gray_img[i, j]) + \
                         (horizontal[1, 2] * gray_img[i, j + 1]) + \
                         (horizontal[2, 0] * gray_img[i + 1, j - 1]) + \
                         (horizontal[2, 1] * gray_img[i + 1, j]) + \
                         (horizontal[2, 2] * gray_img[i + 1, j + 1])

        verticalGrad = (vertical[0, 0] * gray_img[i - 1, j - 1]) + \
                       (vertical[0, 1] * gray_img[i - 1, j]) + \
                       (vertical[0, 2] * gray_img[i - 1, j + 1]) + \
                       (vertical[1, 0] * gray_img[i, j - 1]) + \
                       (vertical[1, 1] * gray_img[i, j]) + \
                       (vertical[1, 2] * gray_img[i, j + 1]) + \
                       (vertical[2, 0] * gray_img[i + 1, j - 1]) + \
                       (vertical[2, 1] * gray_img[i + 1, j]) + \
                       (vertical[2, 2] * gray_img[i + 1, j + 1])

        # Edge Magnitude
        mag = np.sqrt(pow(horizontalGrad, 2.0) + pow(verticalGrad, 2.0))
        newgradientImage_laplachian[i - 1, j - 1] = mag

cv2.imwrite("cartoon_laplachian.jpg", newgradientImage_laplachian)

end = time.time()

print(end-start)

In [ ]:
start = time.time()

#morphologic edge out https://stackoverflow.com/questions/61643039/improving-canny-edge-detection
img = cv2.imread("cartoon.jpg")

# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
threshold = cv2.threshold(gray_img, 180, 255, cv2.THRESH_BINARY)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilate = cv2.morphologyEx(threshold, cv2.MORPH_DILATE, kernel)

diff = cv2.absdiff(dilate, threshold)
# edges = 255-diff
        
# cv2.Canny(frame_webcam, 100, 110, edges=outframe)
cv2.imwrite("cartoon_edges.jpg", diff)

end = time.time()

print(end-start)

In [ ]:
start = time.time()

img2 = cv2.Canny(img, 100, 110)
# cannyEdges = 255-img2

cv2.imwrite('cartoon_canny.jpg', img2)

end = time.time()

print(end-start)

In [95]:
cv2.imwrite("sobel-prewitt.jpg", newgradientImage_sobel-newgradientImage_prewitt)
cv2.imwrite("sobel-laplachian.jpg", newgradientImage_sobel-newgradientImage_laplachian)
cv2.imwrite("prewitt-laplachian.jpg", newgradientImage_prewitt-newgradientImage_laplachian)
cv2.imwrite("sobelmanual-sobelgiven.jpg", newgradientImage_sobel+mag)
cv2.imwrite("cartoon_gray.jpg", cv2.bitwise_not(img))


True

In [ ]:
# look into different gaussian filters x
# different edge out functions
# diferrent operator functions https://medium.com/@nikatsanka/comparing-edge-detection-methods-638a2919476e x